In [ ]:
# Identifying the duplicate School's  
duplicate_data = (school_data[school_data.duplicated(subset=['School Name'], keep = False)])

# Setup variable to hold duplicate school names
duplicate_school_name = duplicate_data['School Name'].unique()

# Create a clean DataFrame by dropping the duplicate Schools
clean_school_df = pd.DataFrame(school_data[school_data["School Name"] != duplicate_school_name[1]])

# Reset index after clean
clean_school_df = clean_school_df.reset_index(drop=True)

In [ ]:
# Identifying the duplicate School's  
duplicate_data = (school_results_df[school_results_df.duplicated(subset=['School'], keep = False)])

# Setup variable to hold duplicate school names
duplicate_school_name = duplicate_data['School'].unique()

# Create a clean DataFrame by dropping the duplicate Schools
clean_school_results_df = pd.DataFrame(school_results_df[school_results_df["School"] != duplicate_school_name[0]])

# Reset index after clean
clean_school_results_df = clean_school_results_df.reset_index(drop=True)

In [ ]:
.str.lower()